## Implementation of Stack, Queues, and Deques

### (1) Stack Abstract Data Type
#### The list class already supports adding an element to the end with the append method, and removing the last element with the pop method